In [ ]:
#[참고] https://jpjp9292.tistory.com/4

**Load Modules**

In [ ]:
# Utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import transforms
import torchvision
! pip install tensorboardX
from tensorboardX import SummaryWriter

# sklearn
from sklearn.metrics import confusion_matrix, classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.8 MB/s eta 0:00:00


**Set Hyperparameters**

In [ ]:
batch_size = 64
#num_epochs = 40     # for VGG11
num_epochs = 25
num_epochs2 = 30    # for VGG16
#learning_rate = 0.0001
learning_rate = 0.01

**Load Data**
- CIFAR10

In [ ]:
# load dataset
train_set = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transforms.ToTensor())
test_set = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [00:10<00:00, 16460382.49it/s]


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


**Mean subtraction of RGB per channel**

In [ ]:
train_meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in train_set]
train_stdRGB = [np.std(x.numpy(), axis=(1,2)) for x, _ in train_set]

train_meanR = np.mean([m[0] for m in train_meanRGB])
train_meanG = np.mean([m[1] for m in train_meanRGB])
train_meanB = np.mean([m[2] for m in train_meanRGB])
train_stdR = np.mean([s[0] for s in train_stdRGB])
train_stdG = np.mean([s[1] for s in train_stdRGB])
train_stdB = np.mean([s[2] for s in train_stdRGB])

print(train_meanR, train_meanG, train_meanB)
print(train_stdR, train_stdG, train_stdB)

0.49139965 0.48215845 0.4465309
0.20220213 0.19931543 0.20086348


**Define the image transformation for data**

In [ ]:
train_transformer = transforms.Compose([transforms.Resize((256,256)),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[train_meanR, train_meanG, train_meanB], std=[train_stdR, train_stdG, train_stdB])])

train_set.transform = train_transformer
test_set.transform = train_transformer

**Define DataLoader**

In [ ]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

# VGG11A
- Create VGG Model A(11 layers)

In [ ]:
# define VGGnet class
class VGGNet(nn.Module):
    def __init__(self) :
        super().__init__()
        self.conv = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
          nn.Linear(in_features=512 * 7 * 7, out_features=4096),
          nn.BatchNorm1d(4096),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5),

          nn.Linear(in_features=4096, out_features=4096),
          nn.BatchNorm1d(4096),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5),

          nn.Linear(in_features=4096, out_features=10),
          nn.BatchNorm1d(10),
          nn.Softmax(dim=1)
        )

        self.apply(self._init_weight_bias)

    def _init_weight_bias(self, module):
        classname = module.__class__.__name__
        if classname == 'Conv2d':
          nn.init.normal_(module.weight, mean=0, std=0.01)
          nn.init.constant_(module.bias, val=0)
        elif classname == 'Linear':
          nn.init.normal_(module.weight, mean=0, std=0.01)
          nn.init.constant_(module.bias, val=0)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
## 변형1 - 6,8번째 컨볼루션 레이어를 5x5로 변경

class VGGNet(nn.Module):
    def __init__(self) :
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=5, padding=2),  # 6번째 컨볼루션 레이어 변경
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=5, padding=2),  # 8번째 컨볼루션 레이어 변경
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Linear(in_features=512 * 7 * 7, out_features=4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=10),
            nn.BatchNorm1d(10),
            nn.Softmax(dim=1)
        )

        self.apply(self._init_weight_bias)

    def _init_weight_bias(self, module):
        classname = module.__class__.__name__
        if classname == 'Conv2d':
            nn.init.normal_(module.weight, mean=0, std=0.01)
            nn.init.constant_(module.bias, val=0)
        elif classname == 'Linear':
            nn.init.normal_(module.weight, mean=0, std=0.01)
            nn.init.constant_(module.bias, val=0)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [ ]:
## 변형2 - max pooling을 overlapping max pooling으로

class VGGNet(nn.Module):
    def __init__(self) :
        super().__init__()
        self.conv = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=1),  # Overlapping Max Pooling

          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=1),  # Overlapping Max Pooling

          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=1),  # Overlapping Max Pooling

          nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=1),  # Overlapping Max Pooling

          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=1)  # Overlapping Max Pooling
        )

        self.fc = nn.Sequential(
          nn.Linear(in_features=512 * 14 * 14, out_features=4096),  # Adjusted for larger feature maps
          nn.BatchNorm1d(4096),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5),

          nn.Linear(in_features=4096, out_features=4096),
          nn.BatchNorm1d(4096),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5),

          nn.Linear(in_features=4096, out_features=10),
          nn.BatchNorm1d(10),
          nn.Softmax(dim=1)
        )

        self.apply(self._init_weight_bias)

    def _init_weight_bias(self, module):
        classname = module.__class__.__name__
        if classname == 'Conv2d':
          nn.init.normal_(module.weight, mean=0, std=0.01)
          nn.init.constant_(module.bias, val=0)
        elif classname == 'Linear':
          nn.init.normal_(module.weight, mean=0, std=0.01)
          nn.init.constant_(module.bias, val=0)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

**Set Device and Check Summary**

In [ ]:
use_cuda = torch.cuda.is_available()
print("use_cuda : ", use_cuda)

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
device = torch.device("cuda:0" if use_cuda else "cpu")

net = VGGNet().to(device)

X = torch.randn(size=(3,224,224)).type(FloatTensor)
print(summary(net, (3,224,224)))

use_cuda :  True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 128, 112, 112]          73,856
       BatchNorm2d-6        [-1, 128, 112, 112]             256
              ReLU-7        [-1, 128, 112, 112]               0
         MaxPool2d-8          [-1, 128, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         295,168
      BatchNorm2d-10          [-1, 256, 56, 56]             512
             ReLU-11          [-1, 256, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]         590,080
      BatchNorm2d-13          [-1, 256, 56, 56]             512
             ReLU-14  

**Loss and Optimizer**

In [ ]:
use_cuda = torch.cuda.is_available()
print("use_cuda : ", use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
model = VGGNet().to(device)
criterion = F.cross_entropy
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

use_cuda :  True


**Training Loop**

In [ ]:
writer = SummaryWriter("./vggnet/tensorboard")

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 30 == 0:
            print(f"{batch_idx*len(data)}/{len(train_loader.dataset)}")

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='mean').item()
            writer.add_scalar("Test Loss", test_loss, epoch)
            pred = output.argmax(1)
            correct += float((pred == target).sum())
            writer.add_scalar("Test Accuracy", correct, epoch)

        test_loss /= len(test_loader.dataset)
        correct /= len(test_loader.dataset)
        return test_loss, correct
        writer.close()

In [ ]:
!nvidia-smi

Wed Aug  7 12:01:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              28W /  70W |   1935MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Per-Epoch Activity**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
for epoch in tqdm(range(1, num_epochs + 1)):
    train(model, device, trainloader, optimizer, epoch)
    test_loss, test_accuracy = test(model, device, testloader)
    writer.add_scalar("Test Loss", test_loss, epoch)
    writer.add_scalar("Test Accuracy", test_accuracy, epoch)
    print(f"Processing Result =  Epoch : {epoch}   Loss : {test_loss}   Accuracy : {test_accuracy}")
    writer.close()


  0%|          | 0/25 [00:00<?, ?it/s]

0/50000
1920/50000
3840/50000
5760/50000
7680/50000
9600/50000
11520/50000
13440/50000
15360/50000
17280/50000
19200/50000
21120/50000
23040/50000
24960/50000
26880/50000
28800/50000
30720/50000
32640/50000
34560/50000
36480/50000
38400/50000
40320/50000
42240/50000
44160/50000
46080/50000
48000/50000
49920/50000


  4%|▍         | 1/25 [11:20<4:32:12, 680.54s/it]

Processing Result =  Epoch : 1   Loss : 0.031397978520393374   Accuracy : 0.4613
0/50000
1920/50000
3840/50000
5760/50000
7680/50000
9600/50000
11520/50000
13440/50000
15360/50000
17280/50000
19200/50000
21120/50000
23040/50000
24960/50000
26880/50000
28800/50000
30720/50000
32640/50000
34560/50000
36480/50000
38400/50000
40320/50000
42240/50000
44160/50000
46080/50000
48000/50000
49920/50000


  8%|▊         | 2/25 [22:42<4:21:10, 681.33s/it]

...

Processing Result =  Epoch : 18   Loss : 0.025803852069377898   Accuracy : 0.8191
0/50000
1920/50000
3840/50000
5760/50000
7680/50000
9600/50000
11520/50000
13440/50000
15360/50000
17280/50000
19200/50000
21120/50000
23040/50000
24960/50000
26880/50000
28800/50000
30720/50000
32640/50000
34560/50000
36480/50000
38400/50000
40320/50000
42240/50000
44160/50000
46080/50000
48000/50000
49920/50000


 76%|███████▌  | 19/25 [3:32:37<1:05:38, 656.44s/it]

Processing Result =  Epoch : 19   Loss : 0.025665160632133484   Accuracy : 0.8282
0/50000
1920/50000
3840/50000
5760/50000
7680/50000
9600/50000
11520/50000
13440/50000
15360/50000
17280/50000
19200/50000
21120/50000
23040/50000
24960/50000
26880/50000
28800/50000
30720/50000
32640/50000
34560/50000
36480/50000
38400/50000
40320/50000
42240/50000
44160/50000
46080/50000
48000/50000
49920/50000


 80%|████████  | 20/25 [3:43:28<54:34, 654.89s/it]  

Processing Result =  Epoch : 20   Loss : 0.025692583072185516   Accuracy : 0.8256
0/50000


**Result of VGG Model A**

In [ ]:
print(f"Result of VGG11A = Epoch:{epoch}   Loss:{test_loss}   Accuracy:{test_accuracy}")

NameError: name 'epoch' is not defined

In [ ]:
## GPU 용량 문제로 80%까지만 진행되어서 에러 발생한 것